демо алгоритм для репы

есть несколько сгенерированных меток для данных. 
Для каждой просто учим отдельную модель

Данные генерируются иерархично при помощи N-coalescent, чтобы веса были зависимы


Далее выкидываем часть данных по каждой задаче и обучаем свою модель

In [2]:
import numpy as np
from tabulate import tabulate
from sklearn.linear_model import LinearRegression

In [28]:
def generate_coalescent(K):
    nodes = set(range(K))
    tree_nodes = {i: {"parent":None, "children": [], "delta": -float("inf")} for i in range(K)}
    new_node = -1
    while len(nodes) > 1:
        l, r = np.random.choice(list(nodes), 2, replace=False,)
        nodes.remove(l)
        nodes.remove(r)
        
        new_node = max(tree_nodes.keys()) + 1
        
        nodes.add(new_node)
        tree_nodes[new_node] = {"parent": None, "children": [l,r], "delta": -float("inf")}
        
        tree_nodes[l]["parent"] = new_node
        tree_nodes[r]["parent"] = new_node
        tree_nodes[l]["delta"] = np.random.exponential(1)
        tree_nodes[r]["delta"] = np.random.exponential(1)
    
    tree_nodes['root'] = tree_nodes[new_node]
    return tree_nodes

def get_weights(tree, d, sigma):
    weights = []
    def generate_w(node, tree, d, sigma,):
        if node["parent"] is None:
            node["w"] = np.random.randn(d)
        else:
            node["w"] = np.random.normal(tree[node["parent"]]["w"], sigma * (node["delta"]))
            
        for child in node["children"]:
            generate_w(tree[child], tree, d, sigma)
        if len(node["children"]) == 0:
            weights.append(node["w"])
    generate_w(tree['root'], tree, d, sigma)
    return np.stack(weights)
        


In [29]:
n = 500
K = 5
d = 10
sigma = 0.1
tree = generate_coalescent(K)
weights = get_weights(tree, d, sigma)

X = np.random.randn(n, d)
Y = np.dot(X, weights.T) + np.random.randn(n, K) * 1
print(Y.shape)

(500, 5)


обучим модель для каждой задачи и покажем результаты по каждой задаче

In [30]:
models = []
for i in range(Y.shape[1]):
    model = LinearRegression()
    model.fit(X, Y[:, i])
    models.append(model)

# now we compute r2 and mean squared error
r2 = []
mse = []
for i in range(Y.shape[1]):
    r2.append(models[i].score(X, Y[:, i]))
    mse.append(np.mean((models[i].predict(X) - Y[:, i])**2))

print(tabulate([["R2", *r2], ["MSE", *mse]], headers=["Metric", *[f"Value_{i}" for i in range(K)]], tablefmt="grid"))

+----------+-----------+-----------+-----------+-----------+-----------+
| Metric   |   Value_0 |   Value_1 |   Value_2 |   Value_3 |   Value_4 |
+==========+===========+===========+===========+===========+===========+
| R2       |  0.900486 |  0.877445 |   0.81383 |  0.858428 |  0.893178 |
+----------+-----------+-----------+-----------+-----------+-----------+
| MSE      |  0.969445 |  0.967503 |   1.00958 |  1.04264  |  0.886175 |
+----------+-----------+-----------+-----------+-----------+-----------+
